In [56]:
import pymongo
import pandas as pd
from pandas import Series
client = pymongo.MongoClient('localhost',27017)
db  = client['Ziru_housing']
table = db['Shanghai_Renting']
data = pd.DataFrame(list(table.find()))
del data['_id']

In [57]:
data.head()

,area,district,metro,metro_distance,price,square,title,type
0,42 ㎡06/6层2室1厅,徐汇,距3号线龙漕路站173米,173,4830,42,整租 · 上缝小区2居室-南,整租
1,68.65 ㎡06/6层2室1厅,徐汇,距11号线龙耀路站757米,757,5130,68.65,整租 · 滨江小区2居室-南,整租
2,30.37 ㎡06/6层1室1厅,徐汇,距4号线上海体育馆站236米,236,4690,30.37,整租 · 中原新村1居室-南,整租
3,49.15 ㎡01/6层2室1厅,徐汇,距3号线石龙路站243米,243,4790,49.15,整租 · 东泉小区2居室-南,整租
4,33.17 ㎡04/6层1室1厅,徐汇,距4号线东安路站488米,488,4490,33.17,整租 · 斜土路2200弄1居室-南,整租


In [58]:
data.shape
#data[data['square']=='约13']

(14652, 8)

In [59]:
import re
def square(num):
    # 取薪资均值----------------
    pattern = re.compile('[1-9]\d*.\d*|0.\d*[1-9]\d*')
    temp = num
    res = re.findall(pattern, temp)
    a=0
    for i in res:
            a = float(i)
    square = a
    # 函数返回值
    return square

square_list = []
metro_list = []
for i in range(0,14652):
    square_num = square(data['square'][i])
    metro_num =  square(data['metro_distance'][i])
    square_list.append(square_num)
    metro_list.append(metro_num)
#print(square_list)

In [60]:
sq = Series(square_list)
md = Series(metro_list)
data.insert(6,'S',sq)
data.insert(4,'metro_num',md)

In [61]:
data.head()

,area,district,metro,metro_distance,metro_num,price,square,S,title,type
0,42 ㎡06/6层2室1厅,徐汇,距3号线龙漕路站173米,173,173.0,4830,42,42.00,整租 · 上缝小区2居室-南,整租
1,68.65 ㎡06/6层2室1厅,徐汇,距11号线龙耀路站757米,757,757.0,5130,68.65,68.65,整租 · 滨江小区2居室-南,整租
2,30.37 ㎡06/6层1室1厅,徐汇,距4号线上海体育馆站236米,236,236.0,4690,30.37,30.37,整租 · 中原新村1居室-南,整租
3,49.15 ㎡01/6层2室1厅,徐汇,距3号线石龙路站243米,243,243.0,4790,49.15,49.15,整租 · 东泉小区2居室-南,整租
4,33.17 ㎡04/6层1室1厅,徐汇,距4号线东安路站488米,488,488.0,4490,33.17,33.17,整租 · 斜土路2200弄1居室-南,整租


# 去重

In [62]:
data = data.drop_duplicates()
data.shape

(14453, 10)

# 输出到csv

In [79]:
data.to_csv('housing_shanghai.csv')

# 准备数据

In [66]:
housing = data.copy()

In [67]:
housing = housing[['district','type','metro_num','S','price']]
housing.head()

,district,type,metro_num,S,price
0,徐汇,整租,173.0,42.00,4830
1,徐汇,整租,757.0,68.65,5130
2,徐汇,整租,236.0,30.37,4690
3,徐汇,整租,243.0,49.15,4790
4,徐汇,整租,488.0,33.17,4490


In [68]:
housing_num = housing[['metro_num','S','price']]
housing_num.head()

,metro_num,S,price
0,173.0,42.00,4830
1,757.0,68.65,5130
2,236.0,30.37,4690
3,243.0,49.15,4790
4,488.0,33.17,4490


In [69]:
housing_cat = housing[['district','type']]
housing_cat.head()

,district,type
0,徐汇,整租
1,徐汇,整租
2,徐汇,整租
3,徐汇,整租
4,徐汇,整租


# 流水线

In [70]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Imputer

num_pipeline = Pipeline([
        ('imputer', Imputer(strategy="median")),
        ('std_scaler', StandardScaler())
    ])
#housing_num_tr = num_pipeline.fit_transform(housing_num)

C:\Users\13626\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [71]:
from sklearn.compose import ColumnTransformer

In [77]:
num_attribs = list(housing_num)
cat_attribs = ['district','type']

full_pipeline = ColumnTransformer([
    ("cat", OneHotEncoder(), cat_attribs),
    ("num", num_pipeline, num_attribs),
    ])

housing_prepared = full_pipeline.fit_transform(housing).toarray()

C:\Users\13626\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [78]:
housing_prepared

array([[ 0.        ,  0.        ,  1.        , ..., -1.1958562 ,
         1.15194702,  1.52291143],
       [ 0.        ,  0.        ,  1.        , ..., -0.06452153,
         2.55912427,  1.73173726],
       [ 0.        ,  0.        ,  1.        , ..., -1.07381153,
         0.53785803,  1.42545938],
       ...,
       [ 0.        ,  1.        ,  0.        , ...,  1.51431023,
        -0.51660087, -0.61407286],
       [ 0.        ,  1.        ,  0.        , ..., -0.26986652,
        -1.06574321, -0.59319028],
       [ 0.        ,  1.        ,  0.        , ...,  0.0536487 ,
        -0.61692495, -0.75329008]])